In [2]:
import os
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
import numpy as np
import pandas as pd

In [3]:
data_directory = 'raw_data/'
manifest_path = os.path.join(data_directory, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

sessions = cache.get_session_table()

In [4]:
pathToSave = '/home/amirali/Desktop/Thesis/Codes/unit_data/Data/'

In [31]:
for sessIdx in range(len(sessions.index.values)):
    print("Session: ", sessIdx)
    currSessNum = sessions.index.values[sessIdx]
    currSession = cache.get_session_data(currSessNum)

    if not 'natural_scenes' in currSession.stimulus_names:
        continue
    
    stimTable    = currSession.get_stimulus_table()
    imageTable   = stimTable[stimTable.stimulus_name == 'natural_scenes']
    imageIndices = imageTable.frame.unique()
    nonImageIdx  = np.where(imageIndices == -1.0)
    imageIndices = np.sort(np.delete(imageIndices, nonImageIdx))
    
    startWindow  = np.arange(-0.2, 0.78, 0.01)
    endWindow    = np.arange(-0.18, 0.8, 0.01)
    
    probes            = ['VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam', 'LGd', 'LP']
    currSessionProbes = list(set(list(currSession.units.ecephys_structure_acronym.unique())) & set(probes))
    
    for imageIdx in imageIndices:
        print("Image Stim: ", imageIdx)
        imagePath = os.path.join(pathToSave, str(int(imageIdx)))
        sessionPath = os.path.join(imagePath, str(sessions.index.values[sessIdx]))
        
        if not os.path.exists(imagePath):
            os.makedirs(imagePath)
        
        if not os.path.exists(sessionPath):
            os.makedirs(sessionPath)
            
        thisImageIndices    = np.where(imageTable.frame == imageIdx)[0]
        numImageIterations  = thisImageIndices.shape[0]
        imageStartTime      = imageTable.iloc[thisImageIndices].start_time.values
        for probe in currSessionProbes:
            print("Probe: ", probe)
            probePath = os.path.join(sessionPath, probe)
            if not os.path.exists(probePath):
                os.makedirs(probePath)
                
            probeRows = np.where(currSession.units.ecephys_structure_acronym == probe)
            probeIDs  = currSession.units.ecephys_structure_acronym.iloc[probeRows].index
            imageAvgSpikeValues = np.zeros((numImageIterations, probeIDs.shape[0], startWindow.shape[0]))
            for id in range(len(probeIDs)):
                for timeIdx in range(len(startWindow)):
                    tInit = startWindow[timeIdx]
                    tEnd  = endWindow[timeIdx]
                    for startTime in range(len(imageStartTime)):
                        imageAvgSpikeValues[startTime, id, timeIdx] = np.where((currSession.spike_times[probeIDs[id]] <= imageStartTime[startTime] 
                                                                    + tEnd) & (currSession.spike_times[probeIDs[id]] >= imageStartTime[startTime] 
                                                                    + tInit))[0].shape[0]
            imageAvgSpikeValues = np.mean(imageAvgSpikeValues, axis=0)
            with open(probePath + '/spikes.npy', 'wb') as f:
                np.save(f, imageAvgSpikeValues)

0


KeyboardInterrupt: 

In [30]:
sessions.index.values[sessIdx]

715093703

In [29]:
with open(probePath + '/spikes.npy', 'wb') as f:
    np.save(f, imageAvgSpikeValues)

In [22]:
imageAvgSpikeValues = np.mean(imageAvgSpikeValues, axis=0)

In [26]:
imageAvgSpikeValues.shape

(76, 98)

In [20]:
print(imageAvgSpikeValues[:, 0, 2])

[0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0.
 0. 0.]


In [31]:
imageStartTime

array([5914.68358658, 5960.97227522, 5979.4877275 , 5983.4910775 ,
       6037.5362275 , 6039.78810136, 6049.79647636, 6103.5914075 ,
       6139.6214975 , 6142.62400294, 6197.41976636, 6220.93939908,
       6257.46993136, 6261.97370136, 6313.51676636, 6353.56686916,
       6679.58919022, 6692.85027658, 6749.64772022, 6779.92300386,
       6790.1815775 , 6833.21748294, 6836.72045522, 6852.2334275 ,
       6853.98488636, 6873.50119408, 6879.75640522, 6939.30617294,
       6943.80991022, 6985.59482158, 7037.88849522, 7065.66166658,
       7121.70852658, 7127.21311158, 8027.96551158, 8030.71783022,
       8030.96803386, 8071.2516875 , 8120.79305022, 8156.32272294,
       8176.08926658, 8181.3436275 , 8261.91092522, 8317.95775522,
       8329.71757136, 8389.76772886, 8403.02878522, 8412.53679294,
       8475.83963386, 8492.10319522])

In [27]:
imageStartTime.shape

(50,)

In [21]:
spikes

array([  15.10657208,   15.23963915,   15.41593969, ..., 9638.34191791,
       9638.54071852, 9638.79205263])

In [29]:
print(imageTable.iloc[thisImageIndices])

                         color contrast frame orientation phase  size  \
stimulus_presentation_id                                                
51442                     null     null   0.0        null  null  null   
51627                     null     null   0.0        null  null  null   
51701                     null     null   0.0        null  null  null   
51717                     null     null   0.0        null  null  null   
51933                     null     null   0.0        null  null  null   
51942                     null     null   0.0        null  null  null   
51982                     null     null   0.0        null  null  null   
52197                     null     null   0.0        null  null  null   
52341                     null     null   0.0        null  null  null   
52353                     null     null   0.0        null  null  null   
52572                     null     null   0.0        null  null  null   
52666                     null     null   0.0      

In [27]:
imageStartTime

stimulus_presentation_id
51442    5914.683587
51627    5960.972275
51701    5979.487727
51717    5983.491077
51933    6037.536227
51942    6039.788101
51982    6049.796476
52197    6103.591407
52341    6139.621497
52353    6142.624003
52572    6197.419766
52666    6220.939399
52812    6257.469931
52830    6261.973701
53036    6313.516766
53196    6353.566869
53300    6679.589190
53353    6692.850277
53580    6749.647720
53701    6779.923004
53742    6790.181577
53914    6833.217483
53928    6836.720455
53990    6852.233427
53997    6853.984886
54075    6873.501194
54100    6879.756405
54338    6939.306173
54356    6943.809910
54523    6985.594822
54732    7037.888495
54843    7065.661667
55067    7121.708527
55089    7127.213112
66132    8027.965512
66143    8030.717830
66144    8030.968034
66305    8071.251687
66503    8120.793050
66645    8156.322723
66724    8176.089267
66745    8181.343627
67067    8261.910925
67291    8317.957755
67338    8329.717571
67578    8389.767729
67631    

In [13]:
startWindow.shape

(95,)